In [1]:
import datetime as dt
import pyarrow.feather as feather
import json
import pandas as pd
import pprint
import praw

from google.cloud import storage
from psaw import PushshiftAPI

# Display options
pd.set_option('display.max_columns', 50)
pd.set_option('max_colwidth', 400)

In [2]:
# Import client id and secret to use with PRAW
with open("keys.json", "r") as credentials:
    data = json.load(credentials)
    c_id = data["client_id"]
    c_secret = data["client_secret"]
    u = data["username"]
    p = data["password"]

# Initialise Google Cloud Storage

In [27]:
# Create a client
client = storage.Client()
print("Client created using default project: {}".format(client.project))

# List available buckets
buckets = client.list_buckets()

# List bucket metadata
print("Buckets in {}:".format(client.project))
for item in buckets:
    print("\t Bucket name: {}".format(bucket.name))
    print("\t Bucket location: {}".format(bucket.location))
    print("\t Bucket storage class: {}".format(bucket.storage_class))

Client created using default project: resolute-button-311208
Buckets in resolute-button-311208:
	 Bucket name: dulshani_bucket
	 Bucket location: EUROPE-WEST2
	 Bucket storage class: STANDARD


In [28]:
# Set bucket for data storage
bucket = client.get_bucket('dulshani_bucket')

In [42]:
# Check current data storage utilisation in bucket
blobs = bucket.list_blobs()

print("Blobs in {}:".format(bucket.name))
for item in blobs:
    print("Name: " + item.name)
    print("\t ID: {}".format(item.id))
    print("\t Size: {} MB".format(item.size*0.000001))
    print("\t Content type: {}".format(item.content_type))
    print("\t Public URL: {}".format(item.public_url))
    print("\n")

Blobs in dulshani_bucket:
Name: changemyview
	 ID: dulshani_bucket/changemyview/1623789454304295
	 Size: 42.47897 MB
	 Content type: application/octet-stream
	 Public URL: https://storage.googleapis.com/dulshani_bucket/changemyview




# Data Collection

### Initialisation

In [5]:
# Initialise PRAW instance
reddit = praw.Reddit(
    client_id = c_id,
    client_secret = c_secret,
    user_agent = "my user agent",
    username = u,
    password = p,
)

# check if read only access
# print(reddit.read_only)

# Initialise PSAW instance
api = PushshiftAPI(reddit)

### Function for Data Collection

In [12]:
# Set start and end dates for data retrieval
start_epoch = int(dt.datetime(2018, 4, 1).timestamp())
end_epoch = int(dt.datetime(2021, 4, 30).timestamp())

In [37]:
# Function to retrieve and store a number of posts from a given subreddit
def get_posts(subreddit, start_epoch, end_epoch, num_posts = None):
    try:
        
        # Extract submissions via psaw and praw
        if num_posts is None:
            # Download all posts    
            submissions = api.search_submissions(subreddit = subreddit,
                                             after = start_epoch,
                                             before = end_epoch)
        
        else:
            # Download specified number of posts
            submissions = api.search_submissions(subreddit = subreddit, 
                                             limit = num_posts, 
                                             after = start_epoch,
                                             before = end_epoch)
        
        
        # Convert downloaded data to a dataframe
        df = pd.DataFrame( [vars(submission) for submission in submissions] )
        
        # Extract the required columns
        cleaned_df = df[['subreddit', 
                         'title', 
                         'selftext', 
                         'created_utc', 
                         'author', 
                         'link_flair_text', 
                         'num_comments', 
                         'score', 
                         'upvote_ratio', 
                         'url']]
        
        # Convert dates to datetime
        cleaned_df['created_utc'] = pd.to_datetime(cleaned_df['created_utc'], unit = 's')
        
        # Convert subreddit and author objects to string 
        cleaned_df['subreddit'] = cleaned_df['subreddit'].astype(str)
        cleaned_df['author'] = cleaned_df['author'].astype(str)
        
        # Save as a feather - heh!
        path = 'data/' + subreddit
        feather.write_feather(cleaned_df, path)
        print("File saved locally")
        
        # Save to storage bucket
        blob_name = subreddit
        blob = bucket.blob(blob_name)
        source_file_name = "data/" + blob_name
        blob.upload_from_filename(source_file_name)
        print("File uploaded to {}.".format(bucket.name))
        
        return(cleaned_df)
    
    
    except Exception as e:
        print('Error:', e)

### Retrieve submissions for CMV

In [40]:
# Get all posts for CMV in the given time period
cmv_df = get_posts('changemyview', start_epoch, end_epoch).sort_values(by = 'score', ascending = False)
cmv_df.head(5)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

File saved locally
File uploaded to dulshani_bucket.


,subreddit,title,selftext,created_utc,author,link_flair_text,num_comments,score,upvote_ratio,url
45877,changemyview,CMV: Mike Bloomberg's campaign is proof that the ultra wealthy in the US can afford a higher tax rate with no ill effect on them,"Mike Bloomberg recently dropped out of the 2020 Presidential race after spending over $500 million dollars of his own money on his campaign. Even with this loss, he is still worth over $55 billion dollars.\n\n\nI believe that this effortless spending on Bloombergs part showcases the reality that is wealth inequality between the ultra rich and working class. While I do not believe the rich shou...",2020-03-05 18:22:18,boss_454,Delta(s) from OP,3753,65396,0.84,https://www.reddit.com/r/changemyview/comments/fdziov/cmv_mike_bloombergs_campaign_is_proof_that_the/
30287,changemyview,CMV: Kanye West is a shill for president Trump and running to syphon off young voters from voting for Biden.,"Kanye West is a shill for president Trump and running to syphon off young voters from voting for Biden. First, he may have an excentric take on reality but can't honestly believe that he has a chance of winning. Second, if he truly wanted to be president, he would have started his campaign last year when he still had a chance to get on state ballots. Third, he has been an avid defender of Trum...",2020-07-05 16:25:43,Psyworld,Delta(s) from OP,2080,54485,0.79,https://www.reddit.com/r/changemyview/comments/hlpd7d/cmv_kanye_west_is_a_shill_for_president_trump_and/
14962,changemyview,CMV: Every candidate for a federal position in the United States of America Government should take the same Civics test that immigrants need if they want to become citizens.,[removed],2020-11-15 18:03:51,thehistoryuniversity,Removed - Submission Rule B,1165,49222,0.86,https://www.reddit.com/r/changemyview/comments/juq86c/cmv_every_candidate_for_a_federal_position_in_the/
307,changemyview,CMV: Most Americans who oppose a national healthcare system would quickly change their tune once they benefited from it.,"I used to think I was against a national healthcare system until after I got out of the army. Granted the VA isn't always great necessarily, but it feels *fantastic* to walk out of the hospital after an appointment without ever seeing a cash register when it would have cost me potentially thousands of dollars otherwise. It's something that I don't think just veterans should be able to experien...",2021-04-27 15:30:56,CrashRiot,Delta(s) from OP,6951,44820,0.81,https://www.reddit.com/r/changemyview/comments/mzr23d/cmv_most_americans_who_oppose_a_national/
21773,changemyview,CMV: Donald Trump has not made a single lasting positive impact on the USA during his term as president.,"I write this because I am tired of the wild goose chase that is finding examples of his success. Anything surrounding Donald Trump is shrouded in divisive language and biased opinions. Liberals will have you believe he has done nothing, while conservatives will tout his unlimited success. I must be missing it, because any time I try to research into a topic I get lost in opinion and bias. \n\n...",2020-09-10 14:11:37,farm_sauce,Delta(s) from OP,5212,36281,0.74,https://www.reddit.com/r/changemyview/comments/iq41dt/cmv_donald_trump_has_not_made_a_single_lasting/


In [41]:
cmv_df.describe()

,num_comments,score,upvote_ratio
count,107759.000000,107759.000000,107759.000000
mean,36.448733,74.884548,0.768652
std,117.319998,782.396455,0.260082
min,0.000000,0.000000,0.040000
25%,0.000000,1.000000,0.550000
50%,3.000000,1.000000,0.870000
75%,39.000000,3.000000,1.000000
max,7736.000000,65396.000000,1.000000


### Retrieve submissions for Against Hate Subreddits

In [ ]:
# Get all posts for CMV since inception to current date
start_epoch_ahs = int(dt.datetime(2015, 6, 1).timestamp())
end_epoch_ahs = int(dt.datetime(2021, 6, 15).timestamp())
against_hate_df = get_posts('AgainstHateSubreddits', start_epoch_ahs, end_epoch_ahs)
against_hate_df.describe()
against_hate_df.head(5)

### Retrieve submissions for Polarised Subreddits

In [ ]:
# List of required subreddits for the analysis
subreddits = ['FemaleDatingStrategy', 'MGTOW2']

In [38]:
# Retrieve submissions for subreddts
for subreddit in subreddits:
    get_posts(subreddit)

File saved locally
File uploaded to dulshani_bucket.


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

Blob changemyview deleted.


In [ ]:
against_hate_df.head(5)

In [ ]:
df = pd.DataFrame([ vars(post) for post in reddit.subreddit("AgainstHateSubreddits").top("year", limit=10)])
df.head(3)

In [ ]:
# Experimenting with the Pushshift API
# https://github.com/dmarx/psaw/issues/78
api = PushshiftAPI()
gen = api.search_submissions(**{"subreddit": "AgainstHateSubreddits", "selftext:not": "[removed]|[deleted]",
                               "filter": "['title', 'selftext', 'created_utc', 'author', 'link_flair_text', 'num_comments', 'score', 'upvote_ratio', 'url']"
                               })

In [ ]:
df = pd.DataFrame([obj.d_ for obj in gen])
df.shape

In [ ]:
df.head(2)

In [ ]:
# Group by flair, aggregate and order by count
df.groupby(['link_flair_text'])['title'].agg(['count']).sort_values(ascending = False)

### Useful Functions

In [39]:
# Delete a blob
blob = client.get_bucket('dulshani_bucket').get_blob('changemyview')
blob.delete()

print("Blob {} deleted.".format(blob.name))

Blob changemyview deleted.


In [ ]:
# Function to retrieve and store a number of posts from a given subreddit
def get_posts(subreddit, num_posts = 100):
    try:
        # Extract posts and save to dataframe
        df = pd.DataFrame([ vars(post) for post in reddit.subreddit(subreddit).top("year", limit = num_posts) ])
        # Extract the required columns
        cleaned_df = df[['title', 'selftext', 'created_utc', 'author', 'link_flair_text', 'num_comments', 'score', 'upvote_ratio', 'url']]
        # Convert author from Redditor to string object
        cleaned_df.loc[:,'author'] = cleaned_df.loc[:,'author'].astype(str)
        # Save as a feather - heh!
        path = 'data/' + subreddit
        feather.write_feather(cleaned_df, path)
    except Exception as e:
        print('Error:', e)
        
# TODO:
# CMV: Also whether the post includes a delta (Flairs should do this)